In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

# Cinemática Inversa

In [2]:
from kinematicsrobotics.datahandler import Save, Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics

## Base de dados

In [3]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe(r'src\data\ready\dataset-radius-1cm.csv')
dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,roll,pich,yaw
0,0,0,12,0,0,53.213314,-2.477121e-15,2.515179,3.141593e+00,-1.361357,2.752040e-16
1,0,12,12,0,0,53.606657,-1.789652e-15,13.742410,2.378531e+00,-1.570796,7.630613e-01
2,0,12,24,0,0,52.819970,-3.124530e-15,6.257590,3.141593e+00,-1.361357,2.371518e-16
3,0,12,24,12,0,53.213314,-2.915404e-15,10.000000,2.356194e+00,-1.570796,7.853982e-01
4,0,12,36,12,0,51.657132,-4.201082e-15,2.678740,3.141593e+00,-1.361357,2.500234e-16
...,...,...,...,...,...,...,...,...,...,...,...
6884,120,120,120,60,0,-9.000000,1.558846e+01,41.176915,-2.026424e-16,-0.523599,-1.047198e+00
6885,120,120,120,72,0,-7.281153,1.261133e+01,42.707475,-1.899993e-16,-0.314159,-1.047198e+00
6886,120,120,120,84,0,-5.440756,9.423666e+00,43.489851,-1.843716e-16,-0.104720,-1.047198e+00
6887,120,120,120,96,0,-3.559244,6.164791e+00,43.489851,-1.843716e-16,0.104720,-1.047198e+00


In [4]:
size_train,size_val,size_test = 0.7, 0.2, 0.1

In [5]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [6]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelos

In [7]:
mlp = Model.mlp_regressor()
mlp.model


MLPRegressor(early_stopping=True, max_iter=1000, n_iter_no_change=5,
             random_state=42)

## Estimação dos hiperparâmetros

In [8]:
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, GridSearchCV
from pandas import DataFrame
from itertools import product

class ParameterOptimizer:
    def __init__(self, *, model: Model, x_train, y_train, size_train = 0.7, size_val =  0.1, size_test = 0.2, n_splits: int = 4) -> None:
        self._model = model
        self._x = x_train
        self._y = y_train
        self.size_validation(size_train = size_train, 
                             size_val = size_val,
                             size_test = size_test
        )
        self.holdout(n_splits = n_splits)
    
    # Método público que define a divisão dos dados de treino e validação
    def size_validation(self, *, size_train, size_val, size_test):
        self._size_val = (1 - size_train/(size_train+size_val))
        self._size_train = 1 - self._size_val
        self._size_test = size_test

    # Validação cruzada hold out
    def holdout(self, *, n_splits = 4):
        self._n_splits = n_splits
        
        self._cv = ShuffleSplit(n_splits=n_splits, 
                                test_size = self._size_val, 
                                random_state=42
        )
    
    


    
class ParameterSearchMLP(ParameterOptimizer):
    def __init__(self, *, min_neurons: int, max_neurons: int, num_layers: int, step: int, **kw) -> None:
        self._min_neurons = min_neurons
        self._max_neurons = max_neurons
        self._num_layers = num_layers
        self._step = step
        super().__init__(**kw)

    def RandomizedSearch(self,*, scoring = 'neg_mean_squared_error', n_iter):
        # Configura os parâmetros da técnica de otimização 
        random_search = RandomizedSearchCV(estimator = self._model.model, 
                                           param_distributions = self.param_grid, 
                                           scoring = scoring, 
                                           cv = self._cv, 
                                           n_iter = n_iter, 
                                           random_state = 42, 
                                           return_train_score = True,
                                           verbose = 1
        )
        
        # Treina os modelos
        random_search.fit(self._x, self._y)

        # DataFrame que armazena os resultado dos hiperparâmetros
        history = DataFrame(random_search.cv_results_)

        # Melhor hiperparâmetro
        best_estimator = random_search.best_estimator_

        return history,best_estimator
    
    def parameter(self, *, activation):
        param_grid = []
        for layers in self._num_layers:
            hidden_layer = self.space_hidden(layers)
            param = { 
                'hidden_layer_sizes': hidden_layer,
                'activation': activation
            }
            param_grid.append(param)
        self.param_grid = param_grid
            

    def space_hidden(self, layers):    
        # Gera a lista de possíveis números de neurônios em cada camada
        possible_neurons = list(range(self._min_neurons, self._max_neurons + 1, self._step))

        return list(product(possible_neurons, repeat=layers))

    

In [14]:
# Espaço de busca do grid search
min_neurons= 100
max_neurons = 300
step = 50
layers = [1 , 2, 3, 4]
activation =  ['relu', 'tanh']

cv = ParameterSearchMLP(min_neurons = min_neurons, 
                        max_neurons = max_neurons, 
                        num_layers = layers, 
                        step = step,
                        model = mlp, 
                        x_train = x_train,
                        y_train = y_train,
                        n_splits = 3
)
cv.parameter(activation = activation)
history,best_estimator = cv.RandomizedSearch(n_iter = 10)
history


Fitting 3 folds for each of 10 candidates, totalling 30 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_activation,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,13.387535,1.136783,0.011968,0.000815,"(150, 200, 250, 150)",tanh,"{'hidden_layer_sizes': (150, 200, 250, 150), '...",-0.006401,-0.005839,-0.007251,-0.006497,0.000580,5,-0.006249,-0.005138,-0.006433,-0.005940,0.000572
1,16.057029,4.791315,0.014297,0.000469,"(300, 100, 300, 300)",tanh,"{'hidden_layer_sizes': (300, 100, 300, 300), '...",-0.006651,-0.009017,-0.018238,-0.011302,0.004999,10,-0.006360,-0.008967,-0.015958,-0.010428,0.004052
2,7.895805,1.411503,0.006315,0.001244,"(300, 200, 100, 100)",relu,"{'hidden_layer_sizes': (300, 200, 100, 100), '...",-0.004531,-0.003969,-0.004343,-0.004281,0.000234,1,-0.003841,-0.003634,-0.003408,-0.003628,0.000177
3,9.686202,2.396786,0.011970,0.000001,"(200, 300, 150, 300)",tanh,"{'hidden_layer_sizes': (200, 300, 150, 300), '...",-0.009063,-0.009531,-0.014436,-0.011010,0.002430,9,-0.008430,-0.009233,-0.012838,-0.010167,0.001917
4,11.010691,1.488175,0.009312,0.000942,"(150, 200, 300, 100)",tanh,"{'hidden_layer_sizes': (150, 200, 300, 100), '...",-0.005596,-0.006632,-0.005670,-0.005966,0.000472,3,-0.005241,-0.006189,-0.004931,-0.005454,0.000535
5,6.835118,0.177020,0.006307,0.000457,"(150, 150, 200, 100)",tanh,"{'hidden_layer_sizes': (150, 150, 200, 100), '...",-0.007603,-0.007857,-0.008670,-0.008043,0.000455,6,-0.007827,-0.007447,-0.008244,-0.007839,0.000325
6,11.526577,1.368093,0.011635,0.001696,"(100, 300, 150, 300)",tanh,"{'hidden_layer_sizes': (100, 300, 150, 300), '...",-0.009810,-0.009858,-0.010454,-0.010041,0.000293,8,-0.009899,-0.010486,-0.009352,-0.009912,0.000463
7,3.836889,0.694854,0.003325,0.000470,"(200, 200, 150)",relu,"{'hidden_layer_sizes': (200, 200, 150), 'activ...",-0.006532,-0.006397,-0.005742,-0.006224,0.000345,4,-0.005955,-0.005823,-0.004811,-0.005530,0.000511
8,4.473346,0.566718,0.002326,0.000470,"(150, 150, 150, 150)",relu,"{'hidden_layer_sizes': (150, 150, 150, 150), '...",-0.003526,-0.004275,-0.005082,-0.004295,0.000635,2,-0.003219,-0.003472,-0.004033,-0.003574,0.000340
9,7.828199,0.409052,0.012633,0.003084,"(200, 200, 100, 250)",tanh,"{'hidden_layer_sizes': (200, 200, 100, 250), '...",-0.009746,-0.010155,-0.008714,-0.009538,0.000606,7,-0.009863,-0.009418,-0.008096,-0.009125,0.000750


cv_results_
best_estimator_
best_params_

In [ ]:
from sklearn.model_selection import ShuffleSplit,RandomizedSearchCV,GridSearchCV
from itertools import product
from sklearn.metrics import mean_squared_error,r2_score
from pandas import DataFrame

# Calcula a porcentagem relativa dos dados de treino e validação
def size_split(size_train,size_val):
    size_val = (1 - size_train/(size_train+size_val))
    return size_val

# Espaço de busca dos neurônios
def space_hidden(min_neurons, max_neurons, layers, step):
    # Lista que armazena os neurônios
    neuron_space = []
    neuron_combinations = []
    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))
    
    # Crie todas as combinações possíveis
    for num_layers in layers:
        neuron_combinations(list(product(possible_neurons, repeat=num_layers)))
        
    neuron_space.extend(neuron_combinations)
    return neuron_space

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Random Search
def RandomizedSearch(x, y, model, param_grid, scoring,cv, n_iter):
    # Configura os parâmetros da técnica de otimização 
    random_search = RandomizedSearchCV(estimator=model, 
                                       param_distributions=param_grid, 
                                       scoring=scoring, 
                                       cv=cv, 
                                       n_iter=n_iter, 
                                       random_state=42, 
                                       return_train_score=True,
                                       verbose=0)
    
    # Treina os modelos
    random_search.fit(x, y)

    # DataFrame que armazena os resultado dos hiperparâmetros
    history = DataFrame(random_search.cv_results_)

    # Melhor hiperparâmetro
    best_estimator = random_search.best_estimator_

    return history,best_estimator

# Grid Search
def GridSearch(x, y, model, param_grid, scoring, cv):
    # Configura os parâmetros da técnica de otimização 
    gridshearch = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               cv=cv,
                               scoring=scoring,
                               verbose=0,
                               return_train_score=True)
    
    # Treina os modelos
    gridshearch.fit(x, y)
    
    # Armazena os resultado dos hiperparâmetros
    history = DataFrame(gridshearch.cv_results_)
    
    best_estimator = gridshearch.best_estimator_

    return history,best_estimator

# Validação cruzada hold out
def holdout(n_splits,test_size):
    return ShuffleSplit(n_splits=n_splits, 
                      test_size=test_size, 
                      random_state=42)



In [ ]:
def space_hidden(min_neurons, max_neurons, layers, step):    
    # Gera a lista de possíveis números de neurônios em cada camada
    possible_neurons = list(range(min_neurons, max_neurons + 1,step))

    return list(product(possible_neurons, repeat=layers))

# Parâmetro da rede mlp
def parameter_mlp(min_neurons,max_neurons,layers,step,func_act): 
    # Dicionário que armazena os hiperparâmetros
    param_grid = {
        'hidden_layer_sizes': space_hidden(min_neurons, max_neurons,layers,step),
        'activation': func_act
    }

    return param_grid

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = 2
func_act =  ['relu', 'tanh']
parameter_mlp(min_neurons,max_neurons,layers,step,func_act)

In [ ]:
# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)

len(param_grid['hidden_layer_sizes'])

In [ ]:
# Salvar as métricas 
sv = save(path_project[0])
path_data_save =  r'src\data\ready\history.csv'

# Dados de treino e validação
size_val = size_split(size_train,size_val)

# Espaço de busca do grid search
min_neurons= 50
max_neurons = 300
step = 10
layers = [2,3]
func_act =  ['relu', 'tanh']

param_grid = parameter_mlp(min_neurons = min_neurons,
                           max_neurons = max_neurons,
                           layers=layers,
                           step=step,
                           func_act=func_act)


# validação cruzada hold out
n_splits = 5

# Número de amostras selecionadas no espaço de busca
n_iter = 1000

# Técnica de validação
cv = holdout(n_splits,size_val)

# inicializando a rede
mlp = MLP()

# APlicação do grid search
history,best_model = RandomizedSearch(x=x_train, 
                           y=y_train, 
                           model=mlp, 
                           param_grid=param_grid,
                           scoring = 'neg_mean_squared_error', 
                           cv=cv,
                           n_iter = n_iter)


sv.dataframe(history,path_data_save)



### Infomações

In [ ]:
print(f"Número de parametros de camada oculta: {len(param_grid['hidden_layer_sizes'])}")
print(f"Atributos do retorno da gridsearsh: {history.columns}")

In [ ]:
best_model

In [ ]:
history[['mean_test_score','std_test_score','mean_train_score', 'std_train_score','params']]